In [1]:
"""
The goal of this script is to calculate feature importance values for each of the 17 tissue-specific models of TRACEvar
using SHAP pakage. In addition, the script creates the SHAP summary plot for every tissue model (Fig. 3A-B). 
"""

In [ ]:
'------------------------ Update Packages -----------------------------------'
# ! pip install --upgrade shap==0.39.0
# ! pip install --upgrade sklearn==0.23.2
# ! pip install --upgrade pandas==1.1.3
# ! pip install --upgrade numpy==1.19.2



In [ ]:
'------------------------ Imports -------------------------------------------'
import os
import pandas as pd
import numpy as np
import multiprocessing as mp
from time import gmtime, strftime

from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import shap
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.inspection import permutation_importance
import pickle

In [2]:
path = os.path.join('..', '..', 'Data', 'Full_Slim_Dataset_hg37-v1.6.csv')
Variants_data = pd.read_csv(path, engine='python')  # low_memory=False,
print(Variants_data)

path = os.path.join('..', '..', 'Results', 'Best_Parameters', 'Best_Parameters_New_17.csv')
Best_param = pd.read_csv(path, engine='python')#low_memory=False,
print(Best_param)


path = os.path.join('..', '..', 'Data', 'Relevant_Columns_Names_Edited_2.csv')
Relevant_Cols_df = pd.read_csv(path)
overlap_cols = Relevant_Cols_df['Feature'].tolist()
print(overlap_cols)
rename_dict = dict(zip(overlap_cols, Relevant_Cols_df['Feature Name'].tolist()))
overlap_cols_names  = Relevant_Cols_df['Feature Name'].tolist()

       VariationID   OMIMs                                 Manifested_Tissues  \
0           535972  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
1           535875  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
2           535979  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
3           567376  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
4           565912  613254  brain-0,kidney,Skin - Sun Exposed (Lower leg),...   
...            ...     ...                                                ...   
67963       873299     NaN                                                NaN   
67964       873211     NaN                                                NaN   
67965       873240     NaN                                                NaN   
67966       873216     NaN                                                NaN   
67967       915562     NaN                                                NaN   

      #Chr        Pos      

In [3]:

y_columns = Variants_data.columns[Variants_data.columns.str.contains(pat='disease_causing')].tolist()
cols = list(Variants_data)
non_relevant_columns = ['VariationID', 'OMIMs', 'Manifested_Tissues', '#Chr', 'Pos', 'ConsDetail', 'motifEName',
                        'FeatureID', 'GeneID_y', 'GeneName', 'CCDS', 'Intron', 'Exon', 'SIFTcat', 'PolyPhenCat',
                        'bStatistic', 'targetScan', 'dbscSNV-rf_score', 'oAA', 'Ref', 'nAA',
                        'Alt']  
non_relevant_columns = non_relevant_columns + y_columns
print(non_relevant_columns)

relevant_columns = [x for x in cols if (x not in non_relevant_columns) and (x in overlap_cols)]
print(relevant_columns)
Relevant_data = Variants_data[relevant_columns]

def preprocessing_new(Relevant_Data):
    
    "---------------------- One Hot Columns -------------------------"
    
    one_hot_columns = ['Type', 'AnnoType', 'Consequence', 'Domain', 'Dst2SplType'] 

    one_hot = pd.get_dummies(Relevant_Data[one_hot_columns])
    Relevant_Data = Relevant_Data.drop(one_hot_columns, axis=1)
    Relevant_Data = Relevant_Data.join(one_hot)
    
    "---------------------- Missing Values Imputation ---------------"
    
    special_imputation_cols = {'SIFTval':1, 'GC':0.42, 'CpG':0.02, 'priPhCons':0.115, 'mamPhCons':0.079, 'verPhCons':0.094,'priPhyloP':-0.033, 'mamPhyloP':-0.038, 'verPhyloP':0.017, 'GerpN':1.91, 'GerpS':-0.2}
    
    for cl in special_imputation_cols:
        Relevant_Data[cl] = Relevant_Data[cl].fillna(special_imputation_cols[cl])
        
    Relevant_Data.fillna(0, inplace=True)
    
    return Relevant_Data

Relevant_data = preprocessing_new(Relevant_data)
Relevant_data.rename(columns=rename_dict, inplace=True)

['VariationID', 'OMIMs', 'Manifested_Tissues', '#Chr', 'Pos', 'ConsDetail', 'motifEName', 'FeatureID', 'GeneID_y', 'GeneName', 'CCDS', 'Intron', 'Exon', 'SIFTcat', 'PolyPhenCat', 'bStatistic', 'targetScan', 'dbscSNV-rf_score', 'oAA', 'Ref', 'nAA', 'Alt', 'Lung_disease_causing', 'Muscle - Skeletal_disease_causing', 'Skin - Sun Exposed_disease_causing', 'Adipose - Subcutaneous_disease_causing', 'Artery - Aorta_disease_causing', 'Heart - Left Ventricle_disease_causing', 'Artery - Coronary_disease_causing', 'brain-0_disease_causing', 'Liver_disease_causing', 'Nerve - Tibial_disease_causing', 'Colon - Sigmoid_disease_causing', 'kidney_disease_causing', 'Heart - Atrial Appendage_disease_causing', 'Breast - Mammary Tissue_disease_causing', 'Uterus_disease_causing', 'Adipose - Visceral_disease_causing', 'Esophagus - Gastroesophageal Junction_disease_causing', 'Esophagus - Mucosa_disease_causing', 'brain-1_disease_causing', 'Skin - Not Sun Exposed_disease_causing', 'Artery - Tibial_disease_caus

In [ ]:

import ast


def shap_values(y):
    
    tissue = y.replace("_disease_causing", "")
    print('---------------------', tissue, '---------------------')
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    
    if tissue.strip() in Best_param['Tissue'].unique():
        
        best_parameters = Best_param['Best_Parameters'][(Best_param['Dataset'] == 'Full Trace')&(Best_param['Tissue'] == tissue.strip())&(Best_param['ML_Model'] == 'Random Forest')].values[0]
        best_parameters = ast.literal_eval(best_parameters)
        
        model = RandomForestClassifier(**best_parameters)
        y_train = Variants_data[y]
        model.fit(Relevant_data, y_train)

        "------------------------ Shap ---------------------------------------"
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(Relevant_data)
        vals = np.abs(shap_values).mean(0)
        print('vals', len(vals), vals)

        Feature_importance = pd.DataFrame(list(zip(Relevant_data.columns, sum(vals))),columns=['col_name', 'feature_importance_vals'])
        Feature_importance.sort_values(by=['feature_importance_vals'], ascending=False, inplace=True)
        path = os.path.join('..', '..', 'Output', 'SHAP_Importance_Slim2',tissue + '_Shap_Importance_Slim.csv')
        Feature_importance.to_csv(path, index=False)
        print('Shap_Feature importance')
        print(Feature_importance.head())
        
        shap.summary_plot(shap_values[1], Relevant_data, show=False)  # Worning check_additivity=False
        plt.tight_layout()
        path = os.path.join('..', '..', 'Output', 'SHAP_Importance_Slim2',tissue + '_Shap_Importance_Slim.jpg')
        plt.savefig(path)

        
    return tissue, strftime("%Y-%m-%d %H:%M:%S", gmtime())

        

        
def driver_func_shap():
    PROCESSES = 20
    df_list = []
    
    with mp.Pool(PROCESSES) as pool:
        
        results = [pool.apply_async(shap_values, (y,)) for y in y_columns]
    
        for r in results:
            
            results_tuple = r.get(timeout=None)
            print('@', results_tuple[0], ' finished', results_tuple[1])

        
if __name__ == '__main__':
    print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))
    driver_func_shap()    
    
    

    

2022-11-30 13:58:26
--------------------- Skin - Sun Exposed ---------------------
--------------------- Adipose - Subcutaneous ---------------------
--------------------- Muscle - Skeletal ---------------------
--------------------- Lung ---------------------
--------------------- Heart - Left Ventricle ---------------------
--------------------- Artery - Aorta ---------------------
--------------------- Artery - Coronary ---------------------
--------------------- brain-0 ---------------------
--------------------- Nerve - Tibial ---------------------
--------------------- Liver ---------------------
2022-11-30 13:58:27
2022-11-30 13:58:27
2022-11-30 13:58:27
--------------------- kidney ---------------------
--------------------- Colon - Sigmoid ---------------------
--------------------- Heart - Atrial Appendage ---------------------
--------------------- Uterus ---------------------
--------------------- Breast - Mammary Tissue ---------------------
2022-11-30 13:58:27
2022-11-30 

22     CADD | Conservation (mamPhyloP)                34.973840
vals 67968 [[3.41653026e-05 7.35191793e-04 5.36441287e-05 ... 0.00000000e+00
  1.48916207e-06 3.31164187e-06]
 [3.30596043e-06 1.31329111e-03 6.77032037e-05 ... 0.00000000e+00
  3.19386541e-07 2.09638833e-06]
 [1.91970098e-03 7.37980859e-04 6.66090653e-05 ... 0.00000000e+00
  3.41403358e-07 1.41523241e-05]
 ...
 [3.35063342e-05 1.02716005e-03 2.67874677e-07 ... 0.00000000e+00
  5.55392156e-07 9.06015979e-08]
 [1.33808809e-05 7.04896396e-04 8.37705941e-06 ... 0.00000000e+00
  2.57346386e-07 1.04936793e-06]
 [9.81825612e-06 6.56604585e-04 1.55813556e-05 ... 0.00000000e+00
  1.71200647e-07 2.47740168e-07]]
Shap_Feature importance
                              col_name  feature_importance_vals
55     CADD | Pathogenic score (PHRED)               169.607458
54  CADD | Pathogenic score (RawScore)               133.430514
1       CADD | Consequence (ConsScore)                68.225821
23     CADD | Conservation (verPhyloP)       

468  Heart | Expression time (Embryonic, week 13)               110.974048
@ Heart - Left Ventricle  finished 2022-11-30 20:02:52
@ Artery - Coronary  finished 2022-11-30 13:58:27
@ brain-0  finished 2022-11-30 14:11:59
@ Liver  finished 2022-11-30 14:50:20
@ Nerve - Tibial  finished 2022-11-30 18:35:29
@ Colon - Sigmoid  finished 2022-11-30 13:58:27
@ kidney  finished 2022-11-30 14:45:58
@ Heart - Atrial Appendage  finished 2022-11-30 13:58:27
@ Breast - Mammary Tissue  finished 2022-11-30 13:58:27
@ Uterus  finished 2022-11-30 13:58:27
@ Adipose - Visceral  finished 2022-11-30 13:58:27
@ Esophagus - Gastroesophageal Junction  finished 2022-11-30 13:58:27
@ Esophagus - Mucosa  finished 2022-11-30 13:58:27
vals 67968 [[1.09307289e-03 8.37662220e-03 1.02825378e-04 ... 3.47041417e-07
  1.76034499e-04 1.60044757e-05]
 [2.73595197e-04 1.09761479e-02 7.79520059e-06 ... 6.70821843e-07
  5.87874906e-06 2.50329628e-05]
 [1.84577430e-03 8.31523234e-03 1.95749950e-04 ... 3.89234672e-07
  6.16677